In [7]:
import polars as pl
import numpy as np
from datetime import datetime
import seaborn as sns

In [8]:
df = pl.read_ndjson("closed-markets.json", infer_schema_length=None)

## About the data

There looks to be some rows that have liquidity still. This could be residual funds that weren't fully withdrawn after market closure (unsure.)

We have access to the trading volume of a given market, as well as it's ID (conditionID)
If we use the conditionID as the label for a market, we can get the trades on said market using the /trades endpoint. With this, we can join information regarding markets and the trades user did on said market



In [9]:
df.head(10).select('startDate', 'gameStartTime','closedTime','endDate', 'outcomes')
df = df.with_columns([
    pl.col('startDate').cast(pl.Datetime),
    pl.col('endDate').cast(pl.Datetime),
    pl.col('volume').cast(pl.Float64),
]).sort('startDate', descending=True, multithreaded=True, nulls_last=True)

In [ ]:
df.select('startDate', 'closedTime', 'volume', 'outcomePrices', 'id','conditionId').head(10)

startDate,closedTime,volume,outcomePrices,id,conditionId
datetime[μs],str,f64,str,str,str
2025-09-30 18:51:06.520729,"""2025-11-01 06:10:12+00""",13916.996797,"""[""0"", ""1""]""","""582336""","""0x6674545cedce09e0b416c81e2d63…"
2025-09-30 18:51:05.278984,"""2025-11-01 06:15:06+00""",350864.837602,"""[""0"", ""1""]""","""582342""","""0x5cc387a272c1e7b13b4693de26c2…"
2025-09-30 18:51:05.025503,"""2025-11-01 06:16:00+00""",8247.194023,"""[""0"", ""1""]""","""582341""","""0x659abca2a83a9b14f92a1909daf3…"
2025-09-30 18:51:04.771979,"""2025-10-13 18:06:05+00""",3436.951193,"""[""1"", ""0""]""","""582348""","""0x6011a64fc8eb90eca62933a0ee7f…"
2025-09-30 18:51:04.518157,"""2025-11-01 06:15:04+00""",6916.597852,"""[""0"", ""1""]""","""582343""","""0x50d7de726cd9ccf7afbbc57f54db…"
2025-09-30 18:51:00.442807,"""2025-10-30 04:26:35+00""",354028.123146,"""[""1"", ""0""]""","""582340""","""0x59eb371f8827a430ffa7dc0ab800…"
2025-09-30 18:50:59.694451,"""2025-11-01 06:05:24+00""",12736.305879,"""[""0"", ""1""]""","""582337""","""0x0f392c4d6bc5d305f2c94983af29…"
2025-09-30 18:50:59.440527,"""2025-11-01 06:03:24+00""",15992.823464,"""[""0"", ""1""]""","""582344""","""0x5b394c74f40230dda3935260ee88…"
2025-09-30 18:50:57.698684,"""2025-11-01 06:16:58+00""",31297.497315,"""[""0"", ""1""]""","""582339""","""0x8906ef02619ac44492cfdb44781b…"


In [17]:
print(df.select('conditionId').head(1).item())

0x6674545cedce09e0b416c81e2d6372398213bdb75e68a2a77882863dbd5397dc
